# Встановлення та імпорт необхідних модулів

In [63]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.9 MB/s eta 0:00:00


In [102]:
! pip install lightgbm

In [124]:
from sklearn.datasets import fetch_california_housing

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.base import clone
from catboost import CatBoostRegressor

from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Аргументація вибору базових моделей на основі Дерев Рішень

У двох минулих роботах з опрацювання навичок регресії та класифікації даних за допомогою методів машинного навчання було розглянуто два датасети, на яких примітивна модель Desicion Tree Regressor&Classifier показала досить очікувані результати з високими можливостями прогнозування результаів кінцевого таргету, але з частковим перенавчанням.

Тому в цій роботі нашою задачею буде спроба регулювання аріхетектури задля підвищення результативності її роботи за допомогою методів ансамблювання та ансамблю моделей. Очікується підвищення точності прогнозування за рахунок комбінування прогнозів від кількох моделей.

#### **Будуть розглянуті:**
* **Decision Tree (Дерево рішень)**: Наглядний і легко інтерпретований метод машинного навчання, який використовується як для класифікації, так і для регресії. Метод працює шляхом розбиття датасету на менші підмножини на основі запитань, що ставляться про атрибути даних, ефективно створюючи деревоподібну структуру з "листками" (рішеннями) і "гілками" (правилами розділення).

* **Bagging (Bootstrap Aggregating)**: Метод, що полягає в генерації кількох підмножин даних шляхом випадкового відбору з повтореннями з оригінального набору даних. Незалежні моделі навчаються на цих підмножинах паралельно. Прогнози від всіх моделей агрегуються (наприклад, шляхом голосування для класифікації або усереднення для регресії) для отримання кінцевого прогнозу. Bagging допомагає зменшити варіативність і запобігти перенавчанню.

* **Random Forest**: Це спеціалізований тип Bagging, де використовуються дерева рішень як базові моделі. Крім випадкового відбору даних, Random Forest також вводить випадковість у вибір ознак при розщепленні вузлів дерева, що дозволяє додатково зменшити кореляцію між деревами. Це забезпечує ще більше зниження варіативності і покращує загальну точність моделі порівняно з простим Bagging.

* **ExtraTrees** (Extremely Randomized Trees, Екстремально випадкові дерева): Ансамблевий алгоритм машинного навчання, що використовує множину дерев рішень для завдань класифікації та регресії. Він подібний до алгоритму Random Forest, але відрізняється за двома ключовими параметрами, які роблять його "екстремально випадковим":
  1. Вибір розділових точок: При створенні кожного розділу в дереві, замість пошуку найкращої точки розділу серед усіх можливих для кожної ознаки (як це робиться в Random Forest), в ExtraTrees випадковим чином вибирається декілька точок розділу для кожної ознаки, а потім з цих точок вибирається та, що дає найкращий розділ. Цей процес дозволяє знизити варіативність моделі за рахунок більшої випадковості.
  2. Вибір ознак: Подібно до Random Forest, ExtraTrees використовує підмножину ознак для кожного розділу, але процес вибору ознак ще більше випадковий.

* **CatBoost**: Високопродуктивна бібліотека для градієнтного бустингу оптимізована для роботи з категорійними даними. Вона автоматично перетворює категорійні ознаки на числові, зменшуючи ризик перенавчання через техніки як обмеження глибини дерев, регуляризація, та випадкові перестановки.

* **Gradient Boosting**: Метод машинного навчання, що використовується для задач регресії, класифікації та інших, який побудований на принципі послідовного покращення прогнозів за допомогою невеликих дерев рішень. Кожне наступне дерево намагається виправити помилки попередніх, застосовуючи градієнтний спуск для мінімізації втрат (функції втрат). Таким чином, модель поступово, крок за кроком, покращує свою точність.

* **LightGBM**: Є реалізацією алгоритму градієнтного бустингу, розробленою компанією Microsoft. Це ефективний та швидкий алгоритм, який використовує техніку побудови дерева на основі гістограм, що дозволяє йому обробляти великі об'єми даних з меншими вимогами до пам'яті та вищою швидкістю навчання порівняно з іншими алгоритмами бустингу, такими як XGBoost або GradientBoostingClassifier з sklearn.

* **Stacking (Стекінг)**: Використовує концепцію "моделей моделей". У цьому підході кілька різних моделей навчаються незалежно, а їх прогнози використовуються як вхідні дані для "мета-моделі", яка намагається вивчити, як найкраще комбінувати прогнози базових моделей для отримання кінцевого прогнозу. Stacking спрямований на зловживання різноманітності базових моделей для покращення прогнозів, вивчаючи, яким чином помилки одних моделей можуть компенсуватися іншими.



# Регресія


In [2]:
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.DataFrame(housing.target, columns=["Target"])

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
# Перетворення y_train і y_test з pandas DataFrame або Series у одновимірні NumPy масиви
y_train_raveled = y_train.values.flatten()
y_test_raveled = y_test.values.flatten()

#### Функція тренування та оцінки моделі регресії

In [40]:
def estimate(model, X_train, X_test, y_train, y_test):
  # Навчання моделі
  model.fit(X_train, y_train)

  # Передбачення значень для тестового набору
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  # Оцінка моделі
  mse_train = mean_squared_error(y_train, y_train_pred)
  mse_test = mean_squared_error(y_test, y_test_pred)
  mae_train = mean_absolute_error(y_train, y_train_pred)
  mae_test = mean_absolute_error(y_test, y_test_pred)
  r2_train = r2_score(y_train, y_train_pred)
  r2_test = r2_score(y_test, y_test_pred)

  # Виведення результатів
  print("MSE Train:", mse_train, "\nMSE Test:", mse_test)
  print("MAE Train:", mae_train, "\nMAE Test:", mae_test)
  print("R² Train:", r2_train, "\nR² Test:", r2_test)

## Класична модель дерева рішень DecisionTreeRegressor

In [5]:
# Створення моделі дерева рішень
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

# Передбачення значень для тестового набору
y_train_pred = tree_reg.predict(X_train)
y_test_pred = tree_reg.predict(X_test)

# Оцінка моделі
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Виведення результатів
print("MSE Train:", mse_train, "\nMSE Test:", mse_test)
print("MAE Train:", mae_train, "\nMAE Test:", mae_test)
print("R² Train:", r2_train, "\nR² Test:", r2_test)

MSE Train: 1.0357621381535573e-31 
MSE Test: 0.49396854311945243
MAE Train: 4.5936553441370335e-17 
MAE Test: 0.45390448401162786
R² Train: 1.0 
R² Test: 0.6230424613065773


## Bagging на основі моделі дерева рішень

In [43]:
# Створення базової моделі
base_model = DecisionTreeRegressor(max_depth=5)

# Створення ансамблевої моделі з використанням Bagging
bagging_model = BaggingRegressor(estimator=base_model, n_estimators=5, random_state=42, max_samples=0.1, max_features=0.1)

# Навчання моделі
estimate(bagging_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.8858253576574003 
MSE Test: 0.8787613534194602
MAE Train: 0.7315623118413944 
MAE Test: 0.7290898003732517
R² Train: 0.33734306121609803 
R² Test: 0.3293991662060235


#### Після налаштування гіперпараметрів

In [44]:
# Створення базової моделі
base_model = DecisionTreeRegressor(max_depth=10, min_samples_leaf=8, min_samples_split=8)

# Створення ансамблевої моделі з використанням Bagging
bagging_model = BaggingRegressor(estimator=base_model, n_estimators=50, random_state=42, max_samples=0.7, max_features=0.9)

# Навчання моделі
estimate(bagging_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.20453934529076426 
MSE Test: 0.2689382984057167
MAE Train: 0.31109319203032865 
MAE Test: 0.351503073825164
R² Train: 0.8469908145667893 
R² Test: 0.7947676619502859


#### Використання ExtraTreesRegressor як базової моделі


In [45]:
# Створення базової моделі з використанням ExtraTreesRegressor
base_model = ExtraTreesRegressor(n_estimators=20, random_state=42)

# Створення ансамблевої моделі з використанням Bagging
bagging_model = BaggingRegressor(estimator=base_model, n_estimators=20, random_state=42, max_samples=0.8, max_features=0.8)

# Навчання моделі
estimate(bagging_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.04846362489023224 
MSE Test: 0.22266256599784096
MAE Train: 0.14556562971021064 
MAE Test: 0.3175205285428779
R² Train: 0.9637459494306402 
R² Test: 0.8300816236036902


## RandomForest

In [48]:
# Створення моделі RandomForestRegressor
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, max_features=0.8)

# Навчання моделі
estimate(random_forest_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.17630416818019903 
MSE Test: 0.2954913872801403
MAE Train: 0.29678857644985635 
MAE Test: 0.36622193754055243
R² Train: 0.8681126258452431 
R² Test: 0.7745044545735564


#### Після налаштування гіперпараметрів

In [58]:
# Створення моделі RandomForestRegressor
random_forest_model = RandomForestRegressor(n_estimators=50, random_state=42, max_depth=15, max_features=0.8, min_samples_split=10, min_samples_leaf=5)

# Навчання моделі
estimate(random_forest_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.12939315120801131 
MSE Test: 0.2681250997016305
MAE Train: 0.23231759576064917 
MAE Test: 0.33808030645337744
R² Train: 0.903205221279898 
R² Test: 0.7953882305800719


## Catboost

In [65]:
# Ініціалізація моделі CatBoostRegressor
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=100, random_seed=42)

# Навчання моделі
estimate(catboost_model, X_train, X_test, y_train_raveled, y_test_raveled)

0:	learn: 1.0934740	total: 75.3ms	remaining: 1m 15s
100:	learn: 0.4867395	total: 2.57s	remaining: 22.9s
200:	learn: 0.4320149	total: 4.33s	remaining: 17.2s
300:	learn: 0.4020581	total: 5.82s	remaining: 13.5s
400:	learn: 0.3803801	total: 6.91s	remaining: 10.3s
500:	learn: 0.3633580	total: 7.87s	remaining: 7.83s
600:	learn: 0.3488402	total: 8.8s	remaining: 5.84s
700:	learn: 0.3358611	total: 10.4s	remaining: 4.45s
800:	learn: 0.3234759	total: 12.2s	remaining: 3.03s
900:	learn: 0.3126821	total: 14s	remaining: 1.54s
999:	learn: 0.3025414	total: 14.7s	remaining: 0us
MSE Train: 0.09153128427351812 
MSE Test: 0.19244942923493022
MAE Train: 0.21135212643198376 
MAE Test: 0.28701788619115914
R² Train: 0.9315284439361162 
R² Test: 0.8531378886816883


## Stacking

Стекінг (stacking) є методом ансамблювання в машинному навчанні, де прогнози декількох моделей (базових естиматорів) використовуються як вхідні дані для мета-моделі, яка робить кінцевий прогноз. Головна ідея стекінгу полягає в тому, щоб об'єднати сильні сторони різних моделей для покращення загальної продуктивності і точності прогнозування.

### Етапи стекінгу:

1. **Навчання базових моделей**: Спочатку набір базових моделей навчається на повному тренувальному наборі даних.

2. **Генерація мета-ознак**: Кожна базова модель використовується для прогнозування на валідаційному наборі або через крос-валідацію на тренувальному наборі. Прогнози цих моделей стають вхідними даними (мета-ознаками) для наступного етапу.

3. **Навчання мета-моделі**: Мета-модель (також звана моделлю другого рівня) навчається на мета-ознаках, тобто на прогнозах, згенерованих базовими моделями. Ціль мета-моделі — вивчити, як найкраще комбінувати прогнози базових моделей для вироблення точнішого кінцевого прогнозу.

In [62]:
# Визначення базових моделей
base_models = [
    RandomForestRegressor(n_estimators=100, random_state=42),
    GradientBoostingRegressor(n_estimators=100, random_state=42),
    DecisionTreeRegressor(random_state=42),
    LinearRegression()
]

# Навчання базових моделей і зберігання прогнозів як нових ознак
X_train_meta = np.zeros((X_train.shape[0], len(base_models)))
X_test_meta = np.zeros((X_test.shape[0], len(base_models)))

for i, model in enumerate(base_models):
    model.fit(X_train, y_train_raveled)
    X_train_meta[:, i] = model.predict(X_train)
    X_test_meta[:, i] = model.predict(X_test)

# Навчання мета-моделі
meta_model = LinearRegression()
estimate(meta_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.5179331255246699 
MSE Test: 0.5558915986952442
MAE Train: 0.5286283596581934 
MAE Test: 0.5332001304956565
R² Train: 0.6125511913966952 
R² Test: 0.575787706032451


In [68]:
base_models = [
    ('catboost', CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)),
    ('extratrees', ExtraTreesRegressor(n_estimators=100, random_state=42)),
    ('randomforest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gradientboosting', GradientBoostingRegressor(n_estimators=100, random_state=42))
]

meta_model = LinearRegression()

stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5)

# Навчання стекінгу моделей
estimate(stacking_model, X_train, X_test, y_train_raveled, y_test_raveled)

MSE Train: 0.03858550095003229 
MSE Test: 0.24175214651654348
MAE Train: 0.13450533798587114 
MAE Test: 0.31948581090996236
R² Train: 0.9711354504361788 
R² Test: 0.8155139727132571


## Висновки

Аналізуючи результати різних методів регресії, можна зробити наступні висновки:

- **Класична модель дерева рішень (DecisionTreeRegressor)** показала високу точність на тренувальному наборі (R² = 1.0), але значно нижчу на тестовому (R² = 0.623), що свідчить про перенавчання моделі.

- **Bagging на основі дерева рішень** показав помірну ефективність, але значне покращення після налаштування гіперпараметрів, з R² з 0.329 до 0.795 на тестовому наборі, що підкреслює важливість оптимізації моделі.

- **Bagging на основі ExtraTreesRegressor** і **RandomForest** обидва показали дуже хороші результати, зокрема ExtraTreesRegressor вирізнявся високими показниками як на тренувальному, так і на тестовому наборах (R² = 0.964 та 0.830 відповідно).

- **CatBoost** виявився найкращим з усіх індивідуальних моделей, демонструючи найвищу точність на тестовому наборі (R² = 0.853) і хорошу здатність до узагальнення.

- Після налаштування гіперпараметрів, **RandomForest** родемонструвала поліпшення у всіх показниках. Це свідчить про те, що налаштування гіперпараметрів дозволило досягти кращого балансу між здатністю моделі навчатися на даних тренувального набору та її здатністю узагальнювати на даних тестового набору, покращуючи загальну ефективність.

- **Stacking** з базовими моделями, що включають CatBoost, ExtraTreesRegressor, RandomForest і GradientBoostingRegressor, показав високу ефективність, особливо на тренувальному наборі (R² = 0.971), але меншу, хоча і все ще високу, на тестовому наборі (R² = 0.816).

**Висновок**: Серед індивідуальних моделей **CatBoost** виявився найкращим за точністю та здатністю до узагальнення. Водночас, ансамбль використання **Stacking** з передовими моделями, включно з CatBoost, показав найкращі результати, демонструючи високу точність та здатність моделі адаптуватися до різних даних. Оптимізація гіперпараметрів є ключовою для покращення результатів, але вимагає обережності, щоб уникнути перенавчання або зниження загальної ефективності моделі.


# Класифікація

In [69]:
# Зчитування даних
data=pd.read_csv('data.csv')

In [71]:
df=pd.DataFrame(data)

In [72]:
df=df.drop('Unnamed: 32',axis=1)
df=df.drop('id',axis=1)

In [73]:
# Поділ датасету на набір ознак X та набір міток цільової змінної y
X0 = df[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]
y0 = df['diagnosis']

In [76]:
# Перетворення категоріальних міток на числові
label_encoder = LabelEncoder()
y= label_encoder.fit_transform(y0)

In [77]:
# Стандартизція ознак
scaler=StandardScaler()
x=scaler.fit_transform(X0)

In [78]:
X=np.array(x)
Y=np.array(y)

In [79]:
# Поділ датасету на навчальну та тренувальну вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Функція тренування та оцінки моделі класифікації

In [85]:
def estimate(model, X_train, X_test, y_train, y_test):
  # Навчання моделі
  model.fit(X_train, y_train)

  # Передбачення значень для тестового набору
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  # Оцінка моделі
  acc_test = accuracy_score(y_test, y_test_pred)
  acc_train = accuracy_score(y_train, y_train_pred)
  recall_test = recall_score(y_test, y_test_pred)
  recall_train = recall_score(y_train, y_train_pred)
  precision_test = precision_score(y_test, y_test_pred)
  precision_train = precision_score(y_train, y_train_pred)
  f1_test = f1_score(y_test, y_test_pred)
  f1_train = f1_score(y_train, y_train_pred)
  roc_auc = roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1])

  # Виведення результатів
  print("Accuracy Train:", acc_train, "\nAccuracy Test:", acc_test)
  print("Recall Train:", recall_train, "\nRecall Test:", recall_test)
  print("Precision Train:", precision_train, "\nPrecision Test:", precision_test)
  print("F1 Train:", f1_train, "\nF1 Test:", f1_test)
  print("ROC AUC Test:", roc_auc)


## Класична модель дерева рішень DecisionTreeClassifier

In [88]:
# Створення та тренування моделі дерева рішень
dt = DecisionTreeClassifier(random_state=42)

In [89]:
estimate(dt, X_train, X_test, y_train, y_test)

Accuracy Train: 1.0 
Accuracy Test: 0.9298245614035088
Recall Train: 1.0 
Recall Test: 0.9302325581395349
Precision Train: 1.0 
Precision Test: 0.8888888888888888
F1 Train: 1.0 
F1 Test: 0.9090909090909092
ROC AUC Test: 0.9299050114641335


#### Після налаштування гіперпараметрів

In [86]:
# Створення та тренування моделі дерева рішень
dt = DecisionTreeClassifier(random_state=42, criterion='gini', min_samples_leaf=4, min_samples_split=2)

In [87]:
estimate(dt, X_train, X_test, y_train, y_test)

Accuracy Train: 0.967032967032967 
Accuracy Test: 0.9385964912280702
Recall Train: 0.9112426035502958 
Recall Test: 0.8604651162790697
Precision Train: 1.0 
Precision Test: 0.9736842105263158
F1 Train: 0.9535603715170279 
F1 Test: 0.9135802469135803
ROC AUC Test: 0.9534883720930232


## Bagging на основі моделі дерева рішень

In [97]:
# Створення базової моделі дерева рішень
dt = DecisionTreeClassifier(random_state=42, criterion='gini', min_samples_leaf=4, min_samples_split=2)
dt.fit(X_test, y_test)

# Створення моделі бегінгу з дерева рішень як базової моделі
bagging_model = BaggingClassifier(estimator=dt, n_estimators=200, random_state=42)

# Оцінка моделі
estimate(bagging_model, X_train, X_test, y_train, y_test)

Accuracy Train: 0.9692307692307692 
Accuracy Test: 0.9473684210526315
Recall Train: 0.9526627218934911 
Recall Test: 0.9302325581395349
Precision Train: 0.9640718562874252 
Precision Test: 0.9302325581395349
F1 Train: 0.9583333333333333 
F1 Test: 0.9302325581395349
ROC AUC Test: 0.9995086799868982


#### Після налаштування гіперпараметрів

In [99]:
# Ініціалізація моделі бегінгу з деревом рішень як базовим естиматором
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42, criterion='gini', min_samples_leaf=4, min_samples_split=2),
                                  random_state=42, bootstrap = False, bootstrap_features=False, max_features=1.0, max_samples=0.75, n_estimators=50)

# Оцінка моделі
estimate(bagging_model, X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy Train: 0.9824175824175824 
Accuracy Test: 0.9649122807017544
Recall Train: 0.9644970414201184 
Recall Test: 0.9534883720930233
Precision Train: 0.9878787878787879 
Precision Test: 0.9534883720930233
F1 Train: 0.9760479041916168 
F1 Test: 0.9534883720930233
ROC AUC Test: 0.9995086799868982


## Gradient boosting

In [101]:
# Ініціалізація моделі градієнтного бустингу
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Оцінка моделі
estimate(gb, X_train, X_test, y_train, y_test)

Accuracy Train: 1.0 
Accuracy Test: 0.956140350877193
Recall Train: 1.0 
Recall Test: 0.9302325581395349
Precision Train: 1.0 
Precision Test: 0.9523809523809523
F1 Train: 1.0 
F1 Test: 0.9411764705882352
ROC AUC Test: 0.9995086799868982


## LightGBM



In [117]:
# Створення датасету для LightGBM
train_data = lgb.Dataset(X_train, label=y_train, params={'verbose': -1}, free_raw_data=False)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data, params={'verbose': -1}, free_raw_data=False)

# Встановлення параметрів
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Навчання моделі
gbm = lgb.train(params,
                train_data,
                num_boost_round=300,
                valid_sets=[train_data, test_data],
                valid_names=['train', 'eval'])

# Передбачення
y_pred_test = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_train = gbm.predict(X_train, num_iteration=gbm.best_iteration)
y_pred_binary_test = [1 if x > 0.5 else 0 for x in y_pred_test]
y_pred_binary_train = [1 if x > 0.5 else 0 for x in y_pred_train]

In [121]:
  # Оцінка моделі
  acc_test = accuracy_score(y_test, y_pred_binary_test)
  acc_train = accuracy_score(y_train, y_pred_binary_train)
  recall_test = recall_score(y_test, y_pred_binary_test)
  recall_train = recall_score(y_train, y_pred_binary_train)
  precision_test = precision_score(y_test, y_pred_binary_test)
  precision_train = precision_score(y_train, y_pred_binary_train)
  f1_test = f1_score(y_test, y_pred_binary_test)
  f1_train = f1_score(y_train, y_pred_binary_train)
  roc_auc = roc_auc_score(y_test, y_pred_binary_test)

  # Виведення результатів
  print("Accuracy Train:", acc_train, "\nAccuracy Test:", acc_test)
  print("Recall Train:", recall_train, "\nRecall Test:", recall_test)
  print("Precision Train:", precision_train, "\nPrecision Test:", precision_test)
  print("F1 Train:", f1_train, "\nF1 Test:", f1_test)
  print("ROC AUC Test:", roc_auc)

Accuracy Train: 1.0 
Accuracy Test: 0.9736842105263158
Recall Train: 1.0 
Recall Test: 0.9534883720930233
Precision Train: 1.0 
Precision Test: 0.9761904761904762
F1 Train: 1.0 
F1 Test: 0.9647058823529412
ROC AUC Test: 0.9697019325253848


## Stacking


In [126]:
base_models = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('lr', LogisticRegression(random_state=42))
]

meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Оцінка моделі
estimate(stacking_model, X_train, X_test, y_train, y_test)

Accuracy Train: 0.9340659340659341 
Accuracy Test: 0.9473684210526315
Recall Train: 0.8875739644970414 
Recall Test: 0.9302325581395349
Precision Train: 0.9316770186335404 
Precision Test: 0.9302325581395349
F1 Train: 0.9090909090909092 
F1 Test: 0.9302325581395349
ROC AUC Test: 0.9995086799868982


In [127]:
base_models = [
    ('dt', DecisionTreeClassifier(random_state=42, criterion='gini', min_samples_leaf=4, min_samples_split=2)),
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('lr', LogisticRegression(random_state=42))
]

meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Оцінка моделі
estimate(stacking_model, X_train, X_test, y_train, y_test)

Accuracy Train: 0.9604395604395605 
Accuracy Test: 0.956140350877193
Recall Train: 0.893491124260355 
Recall Test: 0.9534883720930233
Precision Train: 1.0 
Precision Test: 0.9318181818181818
F1 Train: 0.9437500000000001 
F1 Test: 0.942528735632184
ROC AUC Test: 0.9995086799868982


## Висновки

Аналізуючи результати застосування різних методів класифікації та їх зміни після налаштування гіперпараметрів, можна зробити наступні висновки:

1. **DecisionTreeClassifier**: Початкові результати показали високу точність на тренувальному наборі даних і досить хорошу на тестовому. Після налаштування гіперпараметрів, точність на тренувальному наборі знизилася, але точність на тестовому наборі злегка покращилася, свідчачи про зменшення перенавчання і кращу здатність узагальнення.

2. **Bagging (на основі DecisionTreeClassifier)**: Показав високу ефективність як до, так і після налаштування гіперпараметрів, з значним покращенням показників після налаштування. Це демонструє, що використання ансамблів може ефективно зменшувати варіативність і покращувати точність прогнозування.

3. **Gradient Boosting**: Продемонстрував високі показники на тренувальному наборі та дуже хорошу точність на тестовому наборі, підтверджуючи ефективність градієнтного бустингу для задач класифікації.

4. **LightGBM**: Виявився найкращим серед окремих моделей, показавши виняткові результати на тестовому наборі. Це підкреслює перевагу використання легких градієнтних бустингових моделей для задач з великими даними.

5. **Stacking**: Початковий stacking показав хороші результати, але після оптимізації базових моделей і включення мета-моделі (RandomForest) якості класифікації було значно покращено. Це підкреслює значення вибору правильного набору базових моделей та мета-моделі у стекінгу для досягнення високої точності прогнозування.

**Загальний висновок**: Кожен метод класифікації має свої переваги та недоліки, а налаштування гіперпараметрів грає ключову роль у покращенні точності та здатності до узагальнення моделей. Ансамблеві методи, такі як bagging і stacking, разом із градієнтним бустингом, зокрема LightGBM, демонструють високу ефективність у задачах класифікації, покращуючи стійкість до перенавчання та підвищуючи загальну точність моделі.